In [ ]:
!pip install requests beautifulsoup4

# Scrap Name & Start - End Date

In [ ]:
!pip install selenium
!apt-get update # for chromedriver
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 16.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphic

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re

# Step 1: Set up Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# Step 2: Visit the webpage
url = 'https://genshin-impact.fandom.com/wiki/Wish/History'
driver.get(url)
time.sleep(3)  # wait for JS to load

# Step 3: Parse with BeautifulSoup after JS renders
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Step 4: Find the banner table
tables = soup.find_all('table', class_='article-table alternating-colors-table sortable jquery-tablesorter')

merged_data = []
for table in tables:
    for row in table.find_all("tr")[1:]:  # Skip header row
        cells = row.find_all("td")
        if len(cells) >= 3:
            # Extract banner info
            link = cells[0].find("a")
            banner_full = link['title'] if link else ''
            banner_name = banner_full.split('/')[0] if '/' in banner_full else banner_full

            # Extract date range
            date_range = cells[2].text.strip()
            date_match = re.findall(r'([A-Za-z]+\s\d{1,2},\s\d{4})', date_range)
            if len(date_match) == 2:
                start_date, end_date = date_match
            else:
                start_date, end_date = '', ''

            # Step 5: Extract 5-star characters inside this banner row
            mini_cards = row.find_all('div', class_='card-container mini-card')
            for card in mini_cards:
                image_container = card.find('span', class_=lambda x: x and 'card-image-container' in x)
                if image_container and 'card-quality-5' in image_container['class']:
                    char_link = card.find('a')
                    char_name = char_link['title'].strip() if char_link else ''

                    # Add one row per 5-star char!
                    merged_data.append({
                        'Banner Name': banner_name,
                        'Start Date': start_date,
                        'End Date': end_date,
                        '5-Star Character': char_name
                    })

# Step 6: Export
df = pd.DataFrame(merged_data)
df.to_excel('genshin_banner_5stars_separated.xlsx', index=False)
print(df.head(50))


                   Banner Name          Start Date            End Date  \
0            Ballad in Goblets  September 28, 2020    October 18, 2020   
1           Epitome Invocation  September 28, 2020    October 18, 2020   
2           Epitome Invocation  September 28, 2020    October 18, 2020   
3              Sparkling Steps    October 20, 2020   November 10, 2020   
4           Epitome Invocation    October 20, 2020   November 10, 2020   
5           Epitome Invocation    October 20, 2020   November 10, 2020   
6        Farewell of Snezhnaya   November 11, 2020   December 01, 2020   
7           Epitome Invocation   November 11, 2020   December 01, 2020   
8           Epitome Invocation   November 11, 2020   December 01, 2020   
9          Gentry of Hermitage   December 01, 2020   December 22, 2020   
10          Epitome Invocation   December 01, 2020   December 22, 2020   
11          Epitome Invocation   December 01, 2020   December 22, 2020   
12         Secretum Secretorum   Decem

In [ ]:
from google.colab import files
df = pd.DataFrame(merged_data)
df.to_excel('genshin_wish_history.xlsx', index=False)
files.download('genshin_wish_history.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cards = soup.find_all('div', class_='card-container mini-card')

card_data = []
for card in cards:
    # Look for the image container span
    image_container = card.find('span', class_=lambda x: x and 'card-image-container' in x)

    # Filter only the ones that have "card-quality-5"
    if image_container and 'card-quality-5' in image_container['class']:
        link = card.find('a')


        character_name = link['title'].strip() if link else ''



        card_data.append({
            'Character Name': character_name})
card_df = pd.DataFrame(card_data)
print(card_df.head(10))

                    Character Name
0                            Venti
1                   Aquila Favonia
2                        Amos' Bow
3                             Klee
4  Lost Prayer to the Sacred Winds
5                Wolf's Gravestone
6                        Tartaglia
7                   Memory of Dust
8                     Skyward Harp
9                          Zhongli


# Scrap Wish Type Two

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = 'https://genshin-impact.fandom.com/wiki/Category:Character_Event_Wishes-2'

# Get the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags with class "category-page__member-link"
links = soup.find_all('a', class_='category-page__member-link')

# Prepare lists to store Name and Start Date
names = []
start_dates = []

# Loop through all links and split the title
for link in links:
    title = link.get('title')
    if '/' in title:
        # Split by the last "/" to handle cases where names also have "/"
        split_title = title.rsplit('/', 1)
        name = split_title[0]
        date = split_title[1]
        names.append(name)
        start_dates.append(date)

# Create DataFrame
df = pd.DataFrame({
    'Name': names,
    'Start Date': start_dates
})

# Display DataFrame
df.head(10)


,Name,Start Date
0,Adrift in the Harbor,2022-01-25
1,Adrift in the Harbor,2023-05-02
2,Azure Excursion,2022-12-27
3,Azure Excursion,2023-11-28
4,Ballad in Goblets,2022-03-30
5,Ballad in Goblets,2022-09-28
6,Ballad in Goblets,2023-10-17
7,Ballad in Goblets,5.5
8,Born of Ocean Swell,2021-11-24
9,Caution in Confidence,2024-06-05


In [ ]:
df.to_excel('genshin_wish_type2.xlsx', index=False)
from google.colab import files
files.download('genshin_wish_type2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cumulative Prob Sheet

In [4]:
import pandas as pd
import numpy as np

# Constants for Character/Standard/Chronicled Event
char_base_rate = 0.006
char_soft_pity_increment = 0.0585
char_soft_pity_start = 73
char_hard_pity = 90

# Constants for Weapon Event
weapon_base_rate = 0.007
weapon_soft_pity_increment = 0.0552
weapon_soft_pity_start = 63
weapon_hard_pity = 80

# Calculate Character/Standard/Chronicled Event soft pity
n_char = np.arange(1, char_hard_pity + 1)
p_n_char = []
for pull in n_char:
    if pull < char_soft_pity_start:
        p = char_base_rate
    else:
        p = min(char_base_rate + char_soft_pity_increment * (pull - (char_soft_pity_start - 1)), 1)
    p_n_char.append(p)

# Calculate Weapon Event soft pity
n_weapon = np.arange(1, weapon_hard_pity + 1)
p_n_weapon = []
for pull in n_weapon:
    if pull < weapon_soft_pity_start:
        p = weapon_base_rate
    else:
        p = min(weapon_base_rate + weapon_soft_pity_increment * (pull - (weapon_soft_pity_start - 1)), 1)
    p_n_weapon.append(p)
# Character, Standard, Chronicled will share the same rates (as per your earlier message)
banners = ['Character', 'Standard', 'Chronicled', 'Weapon']

# Collect rows for each pull and banner
rows = []

# Character/Standard/Chronicled
for pull in n_char:
    if pull < char_soft_pity_start:
        p = char_base_rate
    else:
        p = min(char_base_rate + char_soft_pity_increment * (pull - (char_soft_pity_start - 1)), 1)
    for banner in ['Character', 'Standard', 'Chronicled']:
        rows.append([pull, banner, p])

# Weapon
for pull in n_weapon:
    if pull < weapon_soft_pity_start:
        p = weapon_base_rate
    else:
        p = min(weapon_base_rate + weapon_soft_pity_increment * (pull - (weapon_soft_pity_start - 1)), 1)
    rows.append([pull, 'Weapon', p])

# Create dataframe in requested format
df_formatted = pd.DataFrame(rows, columns=['Pull Number', 'Banner', 'Soft Pity Rate'])

# Save as CSV
df_formatted.to_excel('Cumulative Prob Sheet.xlsx', index=False)
from google.colab import files
files.download('Cumulative Prob Sheet.xlsx')
df_formatted.head(90)  # Show the first 20 rows as preview


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Pull Number,Banner,Soft Pity Rate
0,1,Character,0.006
1,1,Standard,0.006
2,1,Chronicled,0.006
3,2,Character,0.006
4,2,Standard,0.006
...,...,...,...
85,29,Standard,0.006
86,29,Chronicled,0.006
87,30,Character,0.006
88,30,Standard,0.006
